# 1. PROBLEM STATEMENT
To create a multiple classification model that can classify the crime and normal incidents when exposed to such, as one of the 14 classes.

# 2. DATA
The data is taken from Kaggle, named, "UCF Crime Dataset". It is an extensive dataset with huge amount of videos in each of the 14 classes; 13 with anomaly videos and 1 with normal videos. The dataset folder also contains certain .docx files regarding the structure of dataset and a readme.txt file.

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            50Gi       1.6Gi        45Gi       3.0Mi       4.0Gi        48Gi
Swap:             0B          0B          0B


# 3. EVALUATION
The model will be evaluated on a separate test dataset using accuracy, precision, recall, and F1-score to measure its effectiveness in detecting and classifying violent activities.

# 4. FEATURES
Each video will be decomposed into frames that will serve as the model’s primary features.
The extracted features should include spatial information (visual appearance, object positions) and temporal information (motion patterns across frames).
Together, these features enable the CNN–LSTM model to learn both what is happening in a frame and how it evolves over time.

In [ ]:
# Importing Libraries
import cv2
import os
import random
from pathlib import Path
from tqdm import tqdm
import shutil

In [ ]:
# Set Configurations
BASE_DIR= Path("/content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)")

FRAMES_DIR= BASE_DIR/"Frames"
SPLIT_RATIO= 0.8 #rest 20% for testimg
FRAME_SIZE= (224, 224)
FRAME_SKIP= 5

## Source Folders
SOURCE_FOLDER1 = [
    BASE_DIR / "Anomaly-Videos-Part-1"/ "Anomaly-Videos-Part-1",
    BASE_DIR / "Anomaly-Videos-Part-2"/ "Anomaly-Videos-Part-2"]
SOURCE_FOLDER2 = [
    BASE_DIR / "Anomaly-Videos-Part-3"/ "Anomaly-Videos-Part-3",
    BASE_DIR / "Anomaly-Videos-Part-4"/ "Anomaly-Videos-Part-4"]
SOURCE_FOLDER3 = [
    BASE_DIR / "Normal_Videos_for_Event_Recognition",
    BASE_DIR / "Testing_Normal_Videos_Anomaly"/ "Testing_Normal_Videos_Anomaly"]
SOURCE_FOLDER4 = [
    BASE_DIR / "Anomaly-Videos-Part_5"
]
SOURCE_FOLDER5= [
    BASE_DIR / "Explosion",
]

In [ ]:
# helper functions
def extract_frames(video_path, output_folder, frame_skip=FRAME_SKIP):
  """
  Extracts frames from a single video into output folder.
  """
  cap= cv2.VideoCapture(str(video_path))
  if not cap.isOpened():
    print(f"Could not open {video_path}")
    return

  frame_count= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  os.makedirs(output_folder, exist_ok=True)
  frame_idx= 0
  saved= 0

  while True:
    ret, frame= cap.read()
    if not ret:
      break
    if frame_idx % frame_skip == 0:
      frame= cv2.resize(frame, FRAME_SIZE)
      frame_file= output_folder/f"frame_{saved:05d}.jpg"
      cv2.imwrite(str(frame_file), frame)
      saved += 1
    frame_idx += 1
  cap.release()

In [ ]:
# extraction pipeline
def main():
  all_videos= []

  # Collect all .mp4 videos
  for folder in SOURCE_FOLDER1:
    if not folder.exists():
      continue
    for class_dir in folder.iterdir():
      if not class_dir.is_dir():
        continue

      # Normalize class name
      if "Normal" in class_dir.name:
        label= "Normal"
      else:
        label = class_dir.name.strip()

      videos=  list(class_dir.glob("*.mp4"))
      for v in videos:
        all_videos.append((v,label))

  # Group by class and split into train/test
  class_to_videos= {}
  for path, label in all_videos:
    class_to_videos.setdefault(label, []).append(path)

  print(f"\n Found {len(class_to_videos)} classes:")
  for cls, vids in class_to_videos.items():
    print(f"{cls}: {len(vids)} videos")

  # Extract frames into frames/train/ and frames/test/
  for cls, videos in class_to_videos.items():
    random.shuffle(videos)
    split_idx= int(len(videos) * SPLIT_RATIO)
    train_videos= videos[:split_idx]
    test_videos= videos[split_idx:]

    print(f"\n {cls}: {len(train_videos)} train, {len(test_videos)} test")

    # Train set
    for idx, video_path in enumerate(tqdm(train_videos, desc=f"Extracting {cls} (train)")):
      clip_folder= FRAMES_DIR/"train"/cls/f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    # Test set
    for idx, video_path in enumerate(tqdm(test_videos, desc=f"Extracting {cls} (test)")):
      clip_folder= FRAMES_DIR / "test" / cls / f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    print("\n Frame extraction completed successfully!")
    print(f"Frames saved under: {FRAMES_DIR}")

# RUN
if __name__ == "__main__":
  main()


 Found 4 classes:
Abuse: 51 videos
Arrest: 50 videos
Arson: 50 videos
Assault: 50 videos

 Abuse: 40 train, 11 test


Extracting Abuse (test): 100%|██████████| 11/11 [05:30<00:00, 30.05s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Arrest: 40 train, 10 test


Extracting Arrest (test): 100%|██████████| 10/10 [02:15<00:00, 13.53s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Arson: 40 train, 10 test


Extracting Arson (test): 100%|██████████| 10/10 [01:41<00:00, 10.13s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Assault: 40 train, 10 test


Extracting Assault (test): 100%|██████████| 10/10 [01:54<00:00, 11.48s/it]


 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames


In [ ]:
# extraction pipeline
def main():
  all_videos= []

  # Collect all .mp4 videos
  for folder in SOURCE_FOLDER2:
    if not folder.exists():
      continue
    for class_dir in folder.iterdir():
      if not class_dir.is_dir():
        continue

      # Normalize class name
      if "Normal" in class_dir.name:
        label= "Normal"
      else:
        label = class_dir.name.strip()

      videos=  list(class_dir.glob("*.mp4"))
      for v in videos:
        all_videos.append((v,label))

  # Group by class and split into train/test
  class_to_videos= {}
  for path, label in all_videos:
    class_to_videos.setdefault(label, []).append(path)

  print(f"\n Found {len(class_to_videos)} classes:")
  for cls, vids in class_to_videos.items():
    print(f"{cls}: {len(vids)} videos")

  # Extract frames into frames/train/ and frames/test/
  for cls, videos in class_to_videos.items():
    random.shuffle(videos)
    split_idx= int(len(videos) * SPLIT_RATIO)
    train_videos= videos[:split_idx]
    test_videos= videos[split_idx:]

    print(f"\n {cls}: {len(train_videos)} train, {len(test_videos)} test")

    # Train set
    for idx, video_path in enumerate(tqdm(train_videos, desc=f"Extracting {cls} (train)")):
      clip_folder= FRAMES_DIR/"train"/cls/f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    # Test set
    for idx, video_path in enumerate(tqdm(test_videos, desc=f"Extracting {cls} (test)")):
      clip_folder= FRAMES_DIR / "test" / cls / f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    print("\n Frame extraction completed successfully!")
    print(f"Frames saved under: {FRAMES_DIR}")

# RUN
if __name__ == "__main__":
  main()


 Found 6 classes:
RoadAccidents: 150 videos
Robbery: 150 videos
Shooting: 50 videos
Shoplifting: 50 videos
Stealing: 100 videos
Vandalism: 50 videos

 RoadAccidents: 120 train, 30 test


Extracting RoadAccidents (test): 100%|██████████| 30/30 [03:25<00:00,  6.86s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Robbery: 120 train, 30 test


Extracting Robbery (test): 100%|██████████| 30/30 [06:00<00:00, 12.01s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Shooting: 40 train, 10 test


...


 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Stealing: 80 train, 20 test


Extracting Stealing (test): 100%|██████████| 20/20 [06:50<00:00, 20.53s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Vandalism: 40 train, 10 test


Extracting Vandalism (test): 100%|██████████| 10/10 [02:21<00:00, 14.10s/it]


 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames


In [ ]:
# extraction pipeline
def main():
  all_videos= []

  # Collect all .mp4 videos
  for folder in SOURCE_FOLDER3:
    if not folder.exists():
      continue
    for class_dir in folder.iterdir():
      if not class_dir.is_dir():
        continue

      # Normalize class name
      if "Normal" in class_dir.name:
        label= "Normal"
      else:
        label = class_dir.name.strip()

      videos=  list(class_dir.glob("*.mp4"))
      for v in videos:
        all_videos.append((v,label))

  # Group by class and split into train/test
  class_to_videos= {}
  for path, label in all_videos:
    class_to_videos.setdefault(label, []).append(path)

  print(f"\n Found {len(class_to_videos)} classes:")
  for cls, vids in class_to_videos.items():
    print(f"{cls}: {len(vids)} videos")

  # Extract frames into frames/train/ and frames/test/
  for cls, videos in class_to_videos.items():
    random.shuffle(videos)
    split_idx= int(len(videos) * SPLIT_RATIO)
    train_videos= videos[:split_idx]
    test_videos= videos[split_idx:]

    print(f"\n {cls}: {len(train_videos)} train, {len(test_videos)} test")

    # Train set
    for idx, video_path in enumerate(tqdm(train_videos, desc=f"Extracting {cls} (train)")):
      clip_folder= FRAMES_DIR/"train"/cls/f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    # Test set
    for idx, video_path in enumerate(tqdm(test_videos, desc=f"Extracting {cls} (test)")):
      clip_folder= FRAMES_DIR / "test" / cls / f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    print("\n Frame extraction completed successfully!")
    print(f"Frames saved under: {FRAMES_DIR}")

# RUN
if __name__ == "__main__":
  main()


 Found 1 classes:
Normal: 50 videos

 Normal: 40 train, 10 test


Extracting Normal (test): 100%|██████████| 10/10 [00:55<00:00,  5.56s/it]


 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames


In [ ]:
# extraction pipeline
def main():
  all_videos= []

  # Collect all .mp4 videos
  for folder in SOURCE_FOLDER4:
    if not folder.exists():
      continue
    for class_dir in folder.iterdir():
      if not class_dir.is_dir():
        continue

      # Normalize class name
      if "FightingA" in class_dir.name:
        label= "Fight"
      else:
        label = class_dir.name.strip()

      videos=  list(class_dir.glob("*.mp4"))
      for v in videos:
        all_videos.append((v,label))

  # Group by class and split into train/test
  class_to_videos= {}
  for path, label in all_videos:
    class_to_videos.setdefault(label, []).append(path)

  print(f"\n Found {len(class_to_videos)} classes:")
  for cls, vids in class_to_videos.items():
    print(f"{cls}: {len(vids)} videos")

  # Extract frames into frames/train/ and frames/test/
  for cls, videos in class_to_videos.items():
    random.shuffle(videos)
    split_idx= int(len(videos) * SPLIT_RATIO)
    train_videos= videos[:split_idx]
    test_videos= videos[split_idx:]

    print(f"\n {cls}: {len(train_videos)} train, {len(test_videos)} test")

    # Train set
    for idx, video_path in enumerate(tqdm(train_videos, desc=f"Extracting {cls} (train)")):
      clip_folder= FRAMES_DIR/"train"/cls/f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    # Test set
    for idx, video_path in enumerate(tqdm(test_videos, desc=f"Extracting {cls} (test)")):
      clip_folder= FRAMES_DIR / "test" / cls / f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    print("\n Frame extraction completed successfully!")
    print(f"Frames saved under: {FRAMES_DIR}")

# RUN
if __name__ == "__main__":
  main()


 Found 2 classes:
Burglary: 50 videos
Fight: 50 videos

 Burglary: 40 train, 10 test


Extracting Burglary (test): 100%|██████████| 10/10 [02:00<00:00, 12.06s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Fight: 40 train, 10 test


Extracting Fight (test): 100%|██████████| 10/10 [05:26<00:00, 32.62s/it]


 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames


In [ ]:
# extraction pipeline
def main():
  all_videos= []

  # Collect all .mp4 videos
  for folder in SOURCE_FOLDER5:
    if not folder.exists():
      continue
    for class_dir in folder.iterdir():
      if not class_dir.is_dir():
        continue

      # Normalize class name
      if "Normal" in class_dir.name:
        label= "Normal"
      else:
        label = class_dir.name.strip()

      videos=  list(class_dir.glob("*.mp4"))
      for v in videos:
        all_videos.append((v,label))

  # Group by class and split into train/test
  class_to_videos= {}
  for path, label in all_videos:
    class_to_videos.setdefault(label, []).append(path)

  print(f"\n Found {len(class_to_videos)} classes:")
  for cls, vids in class_to_videos.items():
    print(f"{cls}: {len(vids)} videos")

  # Extract frames into frames/train/ and frames/test/
  for cls, videos in class_to_videos.items():
    random.shuffle(videos)
    split_idx= int(len(videos) * SPLIT_RATIO)
    train_videos= videos[:split_idx]
    test_videos= videos[split_idx:]

    print(f"\n {cls}: {len(train_videos)} train, {len(test_videos)} test")

    # Train set
    for idx, video_path in enumerate(tqdm(train_videos, desc=f"Extracting {cls} (train)")):
      clip_folder= FRAMES_DIR/"train"/cls/f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    # Test set
    for idx, video_path in enumerate(tqdm(test_videos, desc=f"Extracting {cls} (test)")):
      clip_folder= FRAMES_DIR / "test" / cls / f"clip_{idx+1:04d}"
      extract_frames(video_path, clip_folder)

    print("\n Frame extraction completed successfully!")
    print(f"Frames saved under: {FRAMES_DIR}")

# RUN
if __name__ == "__main__":
  main()


 Found 2 classes:
Explosion: 49 videos
Shooting: 50 videos

 Explosion: 39 train, 10 test


Extracting Explosion (test): 100%|██████████| 10/10 [01:07<00:00,  6.79s/it]



 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames

 Shooting: 40 train, 10 test


Extracting Shooting (test): 100%|██████████| 10/10 [23:20<00:00, 140.04s/it]


 Frame extraction completed successfully!
Frames saved under: /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames


In [ ]:
import shutil, os
shutil.rmtree(FRAMES_DIR / "train"/ "Normal", ignore_errors=True)

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

IMG_SIZE= (224, 224)
SEQUENCE_LENGTH= 100 #frames per clip

In [ ]:
import tensorflow as tf

In [ ]:
FRAMES_DIR = Path("/content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames")   # where your frames live: /content/frames/train, /content/frames/test
OUT_DIR    = Path("/content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames/ARGUS_TFRECORDS")  # where to save tfrecords (Drive OK, few files)
SEQ_LEN    = 100   # frames per clip (pad/trunc)
IMG_SIZE   = (224,224)  # resize
compress_jpeg = True    # store frames as jpeg bytes inside TFRecord (reduces disk & TFRecord size)

In [ ]:
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def _bytes_feature(b): return tf.train.Feature(bytes_list=tf.train.BytesList(value=[b]))
def _int64_feature(i): return tf.train.Feature(int64_list=tf.train.Int64List(value=[i]))

In [ ]:
# collect classes from train folder
train_dir = FRAMES_DIR / "train"
classes = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])
print("Detected classes:", classes)

# save classes mapping for later
np.save(str(OUT_DIR / "classes.npy"), np.array(classes))

label_encoder = LabelEncoder().fit(classes)

Detected classes: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fight', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']


In [ ]:
def clip_to_tfrecord(clip_path, label_int):
    """Serialize one clip folder to a tf.train.Example"""
    files = sorted([f for f in os.listdir(clip_path) if f.lower().endswith(('.jpg','.jpeg','.png'))])
    frames_bytes = []

    for fname in files[:SEQ_LEN]:
        img = cv2.imread(str(clip_path / fname))
        if img is None:
            continue
        img = cv2.resize(img, IMG_SIZE)
        if compress_jpeg:
            ok, enc = cv2.imencode('.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
            if not ok:
                continue
            frames_bytes.append(enc.tobytes())
        else:
            # raw bytes (float32 is big) -> store uint8 raw
            frames_bytes.append(img.tobytes())

    # pad by repeating last frame if needed
    if len(frames_bytes) == 0:
        return None
    while len(frames_bytes) < SEQ_LEN:
        frames_bytes.append(frames_bytes[-1])

    feature = {
        "label": _int64_feature(int(label_int)),
        "num_frames": _int64_feature(len(frames_bytes)),
        "height": _int64_feature(IMG_SIZE[0]),
        "width": _int64_feature(IMG_SIZE[1]),
        "channels": _int64_feature(3),
        "clip_name": _bytes_feature(str(clip_path.name).encode("utf-8")),
        # frames as repeated bytes; store concatenated with separator? TF supports bytes_list
        "frames": tf.train.Feature(bytes_list=tf.train.BytesList(value=frames_bytes)),
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example

In [ ]:
# function to write TFRecord for one split and one class
def write_class_tfrecord(split, cls):
    class_dir = FRAMES_DIR / split / cls
    if not class_dir.exists():
        print("Missing:", class_dir); return
    out_file = OUT_DIR / f"{split}_{cls}.tfrecord"
    if out_file.exists():
        print(f"Skipping (exists): {out_file}")
        return
    writer = tf.io.TFRecordWriter(str(out_file))
    label_int = int(label_encoder.transform([cls])[0])
    clips = sorted([p for p in class_dir.iterdir() if p.is_dir()])
    print(f"Writing {out_file}  ({len(clips)} clips)")
    for clip in tqdm(clips):
        ex = clip_to_tfrecord(clip, label_int)
        if ex is None:
            continue
        writer.write(ex.SerializeToString())
    writer.close()
    print("Saved:", out_file)

In [ ]:
# Write records class-by-class for both splits
for split in ("train","test"):
    for cls in classes:
        write_class_tfrecord(split, cls)

print("All TFRecords created in:", OUT_DIR)
print("Saved class list to:", OUT_DIR / "classes.npy")

Skipping (exists): /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames/ARGUS_TFRECORDS/train_Abuse.tfrecord
Skipping (exists): /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames/ARGUS_TFRECORDS/train_Arrest.tfrecord
Skipping (exists): /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames/ARGUS_TFRECORDS/train_Arson.tfrecord
Skipping (exists): /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames/ARGUS_TFRECORDS/train_Assault.tfrecord
Skipping (exists): /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Frames/ARGUS_TFRECORDS/train_Burglary.tfrecord
Skipping (exists): /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Dron

## High-level flow:

1. Read TFRecords with tf.data (parse, decode JPEG bytes → frames tensor).

2. Batch & shuffle with tf.data so training reads only small chunks into RAM.

3. Apply light augmentation (optional, frame-wise or temporal).

4. Model: per-frame CNN (feature extractor) wrapped with TimeDistributed → temporal model (LSTM or GRU) → attention → dense classifier.

5. Train with model.fit() on the tf.data datasets, use callbacks (checkpoint, reduce LR).

6. Save final weights and small artifacts (classes.npy) to Drive.

In [ ]:
import tensorflow as tf

In [ ]:
SEQ_LEN = 100
IMG_H, IMG_W = 224, 224
AUTOTUNE= tf.data.AUTOTUNE

In [ ]:
feature_desc= {
    "label": tf.io.FixedLenFeature([], tf.int64),
    "num_frames": tf.io.FixedLenFeature([], tf.int64),
    "height": tf.io.FixedLenFeature([], tf.int64),
    "width": tf.io.FixedLenFeature([], tf.int64),
    "channels": tf.io.FixedLenFeature([], tf.int64),
    "clip_name": tf.io.FixedLenFeature([], tf.string),
    "frames": tf.io.VarLenFeature(tf.string)  # list of jpeg bytes
}

In [ ]:
def _parse_example(serialized):
  ex= tf.io.parse_single_example(serialized, feature_desc)
  frames_sparse= ex["frames"] #SparseTensor of bytes
  frames= tf.sparse.to_dense(frames_sparse, default_value=b'')
  # decode each jpeg
  def decode_fn(b):
    img= tf.image.decode_jpeg(b, channels=3)
    img= tf.image.resize(img, [IMG_H, IMG_W])
    img= tf.cast(img, tf.float32) / 255.0
    return img

  frames= tf.map_fn(decode_fn, frames, dtype= tf.float32)
  frames= tf.reshape(frames, (SEQ_LEN, IMG_H, IMG_W, 3))
  label= tf.cast(ex["label"], tf.int32)
  return frames, label

In [ ]:
# 3. Build train/val datasets with batching and prefetching
def make_dataset(tfrecord_paths, num_classes, batch_size=1, shuffle_buffer=256, training=True):
    ds = tf.data.TFRecordDataset(tfrecord_paths, num_parallel_reads=AUTOTUNE)
    if training:
        ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(_parse_example, num_parallel_calls=AUTOTUNE)
    # OPTIONAL: augment only training data
    if training:
        ds = ds.map(lambda x, y: augment(x, y), num_parallel_calls=AUTOTUNE)
    # convert int label → one-hot
    ds = ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)),
                num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)
    # IMPORTANT for multi-epoch stability
    if training:
      ds = ds.repeat()
    return ds

In [ ]:
def augment(frames, label):

    # apply augmentation per frame (not to entire sequence)
    def aug_single(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.12)
        img = tf.image.random_contrast(img, 0.9, 1.1)
        img = tf.image.random_crop(img, size=[IMG_H-8, IMG_W-8, 3])
        img = tf.image.resize(img, [IMG_H, IMG_W])
        return img

    frames = tf.map_fn(aug_single, frames)   # apply to each frame
    return frames, label

In [ ]:
classes= np.load(OUT_DIR/ "classes.npy")
num_classes= len(classes)
train_paths= [str(p) for p in OUT_DIR.glob("train_*.tfrecord")]
val_paths= [str(p) for p in OUT_DIR.glob("test_*tfrecord")]
batch_size = 1

train_ds = make_dataset(train_paths,num_classes, batch_size=batch_size, training=True)
val_ds   = make_dataset(val_paths, num_classes, batch_size=batch_size, training=False)

print("Train TFRecords:", len(train_paths))
print("Val TFRecords:", len(val_paths))


Instructions for updating:
Use fn_output_signature instead


Train TFRecords: 14
Val TFRecords: 14


## ARGUS Architecture
Pre-filter (Motion Thresholding & Person Count)  --> CNN (mobilenetV2) --> LSTM (BiLSTM) --> Spatial + Temporal Attention --> Dense Layer (final output of prediction)

In [ ]:
SEQ_LEN=100
batch_size=1

In [ ]:
tf.keras.backend.clear_session() #(if needed)

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

def build_argus_model(seq_len=SEQ_LEN, img_size=(IMG_H, IMG_W), num_classes=num_classes, feature_dim= 512):
   # 1) Base CNN: MobileNetV2 (frame-level feature extractor)
  base_cnn= MobileNetV2(
      include_top= False,
      pooling= "avg",
      input_shape= (img_size[0], img_size[1], 3),
      weights= "imagenet"
  )
  base_cnn.trainable= False  # start frozen; fine-tune later

  # 2) Input: sequence of frames
  frames_in= layers.Input(shape=(seq_len, img_size[0], img_size[1], 3), name="frames")

  # 3) TimeDistributed CNN: apply MobileNetV2 to each frame
  x = layers.TimeDistributed(base_cnn, name="frame_cnn")(frames_in)  # (B, T, cnn_feat_dim)

  # project features to lower dimension
  x = layers.TimeDistributed(layers.Dense(feature_dim, activation='relu'), name="frame_fc")(x)

  # 4) Temporal Modelling using BiLSTM
  x = layers.Bidirectional(
       layers.LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2),
        name="bilstm_1")(x)                   # (B, T, 512)

  x = layers.Bidirectional(
       layers.LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2),
        name="bilstm_2")(x)

  # optional projection (to reduce)
  x = layers.TimeDistributed(layers.Dense(512, activation='relu'))(x)

  # 5) Attention over time (Temporal Attention)
  attn_dim= 128

  # learn an attention score for each timestep
  attn_scores= layers.Dense(attn_dim, activation="tanh")(x)  # (B, T, attn_dim)
  attn_scores= layers.Dense(1, activation=None)(attn_scores) # (B, T, 1)

  # turn scores into weights
  attn_weights= layers.Softmax(axis=1, name="attn_weights")(attn_scores)   # (B, T, 1)

  # weighted sum of LSTM outputs
  x = layers.Multiply()([x, attn_weights])                       # (B, T, feat)

  # attention-weighted temporal pooling (NO Lambda cuz it doesn't let us load our best model in phase 2)
  x = layers.Attention(use_scale=True)([x, x])   # (B, T, feat)
  x = layers.GlobalAveragePooling1D(name="attn_pool")(x)  # (B, feat)

  # 6) Classification Model
  x = layers.LayerNormalization()(x)
  x = layers.Dense(256, activation='gelu')(x)
  x = layers.Dropout(0.4)(x)
  x = layers.Dense(128, activation='gelu')(x)
  x = layers.Dropout(0.3)(x)
  out = layers.Dense(num_classes, activation='softmax')(x)


  model = Model(frames_in, out, name="ARGUS_ViolenceDetector")
  return model

model = build_argus_model()
model.summary()

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "ARGUS_ViolenceDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ frames (InputLayer) │ (None, 100, 224,  │          0 │ -                 │
│                     │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_cnn           │ (None, 100, 1280) │  2,257,984 │ frames[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_fc            │ (None, 100, 512)  │    655,872 │ frame_cnn[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm_1            │ (None, 100, 512)  │  1,574,912 │ frame_fc[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm_2            │ (None, 100, 512)  │  1,574,912 │ bilstm_1[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 100, 512)  │    262,656 │ bilstm_2[0][0]    │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100, 128)  │     65,664 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 100, 1)    │        129 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_weights        │ (None, 100, 1)    │          0 │ dense_3[0][0]     │
│ (Softmax)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 100, 512)  │          0 │ time_distributed… │
│                     │                   │            │ attn_weights[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 100, 512)  │          1 │ multiply[0][0],   │
│ (Attention)         │                   │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_pool           │ (None, 512)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 512)       │      1,024 │ attn_pool[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │    131,328 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 14)        │      1,806 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,559,184 (25.02 MB)

 Trainable params: 4,301,200 (16.41 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Training Cell
(PHASE 1: Backbone Frozen)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

In [ ]:
SAVE_DIR= "/content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained"
os.makedirs(SAVE_DIR, exist_ok=True)
checkpoint_path = os.path.join(SAVE_DIR, "argus_best_model.h5")

In [ ]:
# Callbacks
callbacks=[
    ModelCheckpoint(
      checkpoint_path,
      monitor="val_loss",
      save_best_only=True,
      save_weights_only= False,
      verbose=1
 ),
    EarlyStopping(
      monitor="val_loss",
      patience=5,
      restore_best_weights=True
 ),
    ReduceLROnPlateau(
      monitor="val_loss",
      factor=0.3,
      patience=3,
      verbose=1
 ),
    ModelCheckpoint(
      filepath=checkpoint_path.replace(".h5", "_bestacc.h5"),
      monitor="val_accuracy",
      mode="max",
      save_best_only=True,
      save_weights_only=False,
      verbose=1
)
]

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    steps_per_epoch=721,
    validation_steps=191,
    callbacks=callbacks
)

Epoch 1/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1299 - loss: 2.5190
Epoch 1: val_loss improved from inf to 2.79735, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1542s 2s/step - accuracy: 0.1299 - loss: 2.5189 - val_accuracy: 0.1047 - val_loss: 2.7973 - learning_rate: 1.0000e-04
Epoch 2/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1515 - loss: 2.4912
Epoch 2: val_loss improved from 2.79735 to 2.79514, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1029s 1s/step - accuracy: 0.1515 - loss: 2.4911 - val_accuracy: 0.1047 - val_loss: 2.7951 - learning_rate: 1.0000e-04
Epoch 3/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1837 - loss: 2.4054
Epoch 3: val_loss improved from 2.79514 to 2.50456, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1029s 1s/step - accuracy: 0.1838 - loss: 2.4053 - val_accuracy: 0.1728 - val_loss: 2.5046 - learning_rate: 1.0000e-04
Epoch 4/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2621 - loss: 2.3181
Epoch 4: val_loss did not improve from 2.50456
721/721 ━━━━━━━━━━━━━━━━━━━━ 1025s 1s/step - accuracy: 0.2622 - loss: 2.3181 - val_accuracy: 0.2199 - val_loss: 2.5681 - learning_rate: 1.0000e-04
Epoch 5/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2786 - loss: 2.2539
Epoch 5: val_loss did not improve from 2.50456
721/721 ━━━━━━━━━━━━━━━━━━━━ 1014s 1s/step - accuracy: 0.2786 - loss: 2.2538 - val_accuracy: 0.1780 - val_loss: 2.7052 - learning_rate: 1.0000e-04
Epoch 6/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2874 - loss: 2.2178
Epoch 6: val_loss improved from 2.50456 to 2.46040, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_m

721/721 ━━━━━━━━━━━━━━━━━━━━ 1017s 1s/step - accuracy: 0.2874 - loss: 2.2177 - val_accuracy: 0.2251 - val_loss: 2.4604 - learning_rate: 1.0000e-04
Epoch 7/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3040 - loss: 2.1499
Epoch 7: val_loss improved from 2.46040 to 2.42790, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1026s 1s/step - accuracy: 0.3041 - loss: 2.1500 - val_accuracy: 0.2775 - val_loss: 2.4279 - learning_rate: 1.0000e-04
Epoch 8/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3570 - loss: 2.0341
Epoch 8: val_loss improved from 2.42790 to 2.41085, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1018s 1s/step - accuracy: 0.3570 - loss: 2.0342 - val_accuracy: 0.2880 - val_loss: 2.4108 - learning_rate: 1.0000e-04
Epoch 9/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2744 - loss: 2.1357
Epoch 9: val_loss improved from 2.41085 to 2.36650, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1017s 1s/step - accuracy: 0.2745 - loss: 2.1357 - val_accuracy: 0.2827 - val_loss: 2.3665 - learning_rate: 1.0000e-04
Epoch 10/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3553 - loss: 1.9670
Epoch 10: val_loss did not improve from 2.36650
721/721 ━━━━━━━━━━━━━━━━━━━━ 1014s 1s/step - accuracy: 0.3552 - loss: 1.9671 - val_accuracy: 0.2775 - val_loss: 2.8784 - learning_rate: 1.0000e-04


In [ ]:
!free -h

Phase A’s job is:

**“Does the system learn anything meaningful end-to-end?”**

Answer is **"YES".**

(Phase 2: Unfreeze backbone + fine-tune)

In [ ]:
train_paths = [str(p) for p in OUT_DIR.glob("train_*.tfrecord")]
val_paths   = [str(p) for p in OUT_DIR.glob("test_*.tfrecord")]

In [ ]:
def count_examples_safe(tfrecord_paths):
    total = 0

    for p in tfrecord_paths:
        ds = tf.data.TFRecordDataset(p)

        # THIS is the correct way in your TF version
        ds = ds.apply(tf.data.experimental.ignore_errors())

        for _ in ds:
            total += 1

    return total


train_examples = count_examples_safe(train_paths)
val_examples   = count_examples_safe(val_paths)

BATCH_SIZE = 2

STEPS_PER_EPOCH = train_examples // BATCH_SIZE
VAL_STEPS       = val_examples // BATCH_SIZE

print("Train examples:", train_examples)
print("Val examples:", val_examples)
print("Steps per epoch:", STEPS_PER_EPOCH)
print("Val steps:", VAL_STEPS)

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


Train examples: 721
Val examples: 191
Steps per epoch: 360
Val steps: 95


In [ ]:
import os

print("Mounted files in Model_Trained folder:")
!ls "/content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained"

Mounted files in Model_Trained folder:
 argus_best_model.h5  'Model_Trained_(given)'


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

tf.keras.backend.clear_session()
model = load_model(checkpoint_path)

base_cnn = model.get_layer("frame_cnn").layer

# Freeze everything first
for layer in base_cnn.layers:
  layer.trainable=False

# Unfreeze only last N layers
N= 20
for layer in base_cnn.layers[-N:]:
  if not isinstance(layer, BatchNormalization):
    layer.trainable=True

print("Unfrozen last", N, "layers (except BatchNorm)")
model.summary()

model.compile(
    optimizer=Adam(learning_rate=3e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    steps_per_epoch=721,
    validation_steps=191,
    callbacks=callbacks
)

Unfrozen last 20 layers (except BatchNorm)


Model: "ARGUS_ViolenceDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ frames (InputLayer) │ (None, 100, 224,  │          0 │ -                 │
│                     │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_cnn           │ (None, 100, 1280) │  2,257,984 │ frames[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_fc            │ (None, 100, 512)  │    655,872 │ frame_cnn[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm_1            │ (None, 100, 512)  │  1,574,912 │ frame_fc[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm_2            │ (None, 100, 512)  │  1,574,912 │ bilstm_1[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 100, 512)  │    262,656 │ bilstm_2[0][0]    │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100, 128)  │     65,664 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 100, 1)    │        129 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_weights        │ (None, 100, 1)    │          0 │ dense_3[0][0]     │
│ (Softmax)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 100, 512)  │          0 │ time_distributed… │
│                     │                   │            │ attn_weights[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 100, 512)  │          1 │ multiply[0][0],   │
│ (Attention)         │                   │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_pool           │ (None, 512)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 512)       │      1,024 │ attn_pool[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │    131,328 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 14)        │      1,806 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,559,186 (25.02 MB)

 Trainable params: 5,342,480 (20.38 MB)

 Non-trainable params: 1,216,704 (4.64 MB)

 Optimizer params: 2 (12.00 B)

Epoch 1/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2675 - loss: 2.2331
Epoch 1: val_loss did not improve from 2.36650
721/721 ━━━━━━━━━━━━━━━━━━━━ 1600s 2s/step - accuracy: 0.2676 - loss: 2.2330 - val_accuracy: 0.2827 - val_loss: 2.4108 - learning_rate: 3.0000e-05
Epoch 2/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2525 - loss: 2.2625
Epoch 2: val_loss did not improve from 2.36650
721/721 ━━━━━━━━━━━━━━━━━━━━ 1061s 1s/step - accuracy: 0.2525 - loss: 2.2624 - val_accuracy: 0.2880 - val_loss: 2.3795 - learning_rate: 3.0000e-05
Epoch 3/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3013 - loss: 2.2148
Epoch 3: val_loss did not improve from 2.36650
721/721 ━━━━━━━━━━━━━━━━━━━━ 1061s 1s/step - accuracy: 0.3013 - loss: 2.2148 - val_accuracy: 0.2513 - val_loss: 2.4124 - learning_rate: 3.0000e-05
Epoch 4/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3486 - loss: 2.1419
Epoch 4: val_loss did not improve from 2.36650
721/721 ━━━━━━━━━━━━━━━━━━━━ 11

721/721 ━━━━━━━━━━━━━━━━━━━━ 1068s 1s/step - accuracy: 0.3618 - loss: 1.9978 - val_accuracy: 0.3037 - val_loss: 2.3361 - learning_rate: 9.0000e-06
Epoch 8/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4014 - loss: 1.8857
Epoch 8: val_loss improved from 2.33609 to 2.32332, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1069s 1s/step - accuracy: 0.4014 - loss: 1.8857 - val_accuracy: 0.3037 - val_loss: 2.3233 - learning_rate: 9.0000e-06
Epoch 9/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4115 - loss: 1.8408
Epoch 9: val_loss did not improve from 2.32332
721/721 ━━━━━━━━━━━━━━━━━━━━ 1070s 1s/step - accuracy: 0.4115 - loss: 1.8408 - val_accuracy: 0.2932 - val_loss: 2.4763 - learning_rate: 9.0000e-06
Epoch 10/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3852 - loss: 1.8339
Epoch 10: val_loss did not improve from 2.32332
721/721 ━━━━━━━━━━━━━━━━━━━━ 1069s 1s/step - accuracy: 0.3852 - loss: 1.8339 - val_accuracy: 0.2932 - val_loss: 2.4120 - learning_rate: 9.0000e-06


In [ ]:
import os

print("Mounted files in Model_Trained folder:")
!ls "/content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained"

Mounted files in Model_Trained folder:
 argus_best_model_bestacc.h5   argus_best_model.h5  'Model_Trained_(given)'


**Phase 3: Temporal Refinement**

In [ ]:
# Pre-requisite for Phase 3
# Computing class weights
import numpy as np
from collections import Counter
import tensorflow as tf

def get_labels_from_tfrecord(tfrecord_paths):
    labels = []

    raw_dataset = tf.data.TFRecordDataset(tfrecord_paths)

    for raw_record in raw_dataset:
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        label = example.features.feature["label"].int64_list.value[0]
        labels.append(label)

    return labels

labels = get_labels_from_tfrecord(train_paths)

counts = Counter(labels)
total = sum(counts.values())
num_classes = len(counts)

class_weights = {
    i: total / (num_classes * counts[i]) for i in counts
}

print(class_weights)

{1: 1.355357142857143, 2: 1.355357142857143, 3: 1.355357142857143, 4: 1.355357142857143, 5: 1.39010989010989, 6: 1.355357142857143, 7: 1.355357142857143, 8: 0.4517857142857143, 9: 0.4517857142857143, 10: 1.355357142857143, 11: 1.355357142857143, 12: 0.6776785714285715, 13: 1.355357142857143, 0: 1.355357142857143}


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

tf.keras.backend.clear_session()

# Loading best model from Phase 2
model= load_model(checkpoint_path)

# Freezing all the CNN layers/ complete MobileNetV2
base_cnn= model.get_layer("frame_cnn").layer

for layer in base_cnn.layers:
  layer.trainable= False

for layer in base_cnn.layers[-10:]:
  if not isinstance(layer, BatchNormalization):
    layer.trainable= True
# Compile the model
model.compile(
    optimizer= Adam(learning_rate=3e-5),
    loss= "categorical_crossentropy",
    metrics= ["accuracy"]
)

model.summary()

history= model.fit(
    train_ds,
    validation_data= val_ds,
    epochs=10,
    steps_per_epoch=721,
    validation_steps=191,
    class_weight= class_weights,
    callbacks= callbacks
)

Model: "ARGUS_ViolenceDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ frames (InputLayer) │ (None, 100, 224,  │          0 │ -                 │
│                     │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_cnn           │ (None, 100, 1280) │  2,257,984 │ frames[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_fc            │ (None, 100, 512)  │    655,872 │ frame_cnn[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm_1            │ (None, 100, 512)  │  1,574,912 │ frame_fc[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm_2            │ (None, 100, 512)  │  1,574,912 │ bilstm_1[0][0]    │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 100, 512)  │    262,656 │ bilstm_2[0][0]    │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100, 128)  │     65,664 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 100, 1)    │        129 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_weights        │ (None, 100, 1)    │          0 │ dense_3[0][0]     │
│ (Softmax)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 100, 512)  │          0 │ time_distributed… │
│                     │                   │            │ attn_weights[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 100, 512)  │          1 │ multiply[0][0],   │
│ (Attention)         │                   │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_pool           │ (None, 512)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 512)       │      1,024 │ attn_pool[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │    131,328 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 14)        │      1,806 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,559,184 (25.02 MB)

 Trainable params: 5,026,640 (19.18 MB)

 Non-trainable params: 1,532,544 (5.85 MB)

Epoch 1/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3180 - loss: 2.2678
Epoch 1: val_loss improved from inf to 2.43847, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model.h5



Epoch 1: val_accuracy improved from -inf to 0.24607, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model_bestacc.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1617s 2s/step - accuracy: 0.3181 - loss: 2.2675 - val_accuracy: 0.2461 - val_loss: 2.4385 - learning_rate: 3.0000e-05
Epoch 2/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3604 - loss: 2.0789
Epoch 2: val_loss did not improve from 2.43847

Epoch 2: val_accuracy did not improve from 0.24607
721/721 ━━━━━━━━━━━━━━━━━━━━ 1089s 2s/step - accuracy: 0.3604 - loss: 2.0787 - val_accuracy: 0.2094 - val_loss: 2.6328 - learning_rate: 3.0000e-05
Epoch 3/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3829 - loss: 1.9733
Epoch 3: val_loss did not improve from 2.43847

Epoch 3: val_accuracy improved from 0.24607 to 0.25131, saving model to /content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained/argus_best_model_bestacc.h5


721/721 ━━━━━━━━━━━━━━━━━━━━ 1091s 2s/step - accuracy: 0.3829 - loss: 1.9732 - val_accuracy: 0.2513 - val_loss: 2.6178 - learning_rate: 3.0000e-05
Epoch 4/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4058 - loss: 1.7850
Epoch 4: val_loss did not improve from 2.43847

Epoch 4: ReduceLROnPlateau reducing learning rate to 8.999999772640877e-06.

Epoch 4: val_accuracy did not improve from 0.25131
721/721 ━━━━━━━━━━━━━━━━━━━━ 1088s 2s/step - accuracy: 0.4058 - loss: 1.7850 - val_accuracy: 0.2042 - val_loss: 2.6477 - learning_rate: 3.0000e-05
Epoch 5/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4618 - loss: 1.6904
Epoch 5: val_loss did not improve from 2.43847

Epoch 5: val_accuracy did not improve from 0.25131
721/721 ━━━━━━━━━━━━━━━━━━━━ 1089s 2s/step - accuracy: 0.4618 - loss: 1.6904 - val_accuracy: 0.2408 - val_loss: 2.6073 - learning_rate: 9.0000e-06
Epoch 6/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4581 - loss: 1.5205
Epoch 6: val_loss did not imp

721/721 ━━━━━━━━━━━━━━━━━━━━ 1089s 2s/step - accuracy: 0.4581 - loss: 1.5206 - val_accuracy: 0.2565 - val_loss: 2.6413 - learning_rate: 9.0000e-06


In [ ]:
import os

print("Mounted files in Model_Trained folder:")
!ls "/content/drive/MyDrive/DetectionWithDroneModel(ARGUS)/VideoDetectionDataset/Drone_Detection_Dataset(Unzipped Files)/Model_Trained"

Mounted files in Model_Trained folder:
 argus_best_model_bestacc.h5   argus_best_model.h5  'Model_Trained_(given)'
